# Fine Tunning


---
**Model :** Stable-code-instruct-3b

**Link :** [Stable 3B](https://huggingface.co/stabilityai/stable-code-instruct-3b)



In [ ]:
!pip install -q peft
!pip install -q bitsandbytes
import pandas as pd
import torch
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from torch.utils.data import DataLoader, TensorDataset
from peft import LoraConfig, TaskType, get_peft_model
from peft import PeftModel, PeftConfig

2024-08-09 13:05:46.252318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 13:05:46.252454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 13:05:46.386246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login
hf_token = "hf_yIIEeTomS-----------vEskXzlDffCF"
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df=pd.read_csv("/kaggle/input/dataset/final_data.csv")
df=df[:10000]
df.shape

(10000, 1)

In [ ]:
model_name='stabilityai/stable-code-instruct-3b'

 * Load the tokenizer for the Stable Code Instruct model from the **stabilityai/stable-code-instruct-3b** repository.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/9.35k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Set the padding token to the end-of-sequence token (As per the model requirments from special_tokens_map.json file)
tokenizer.pad_token = tokenizer.eos_token
pt_batch=tokenizer(df['text'].to_list(),
                  padding=True,          # Pad sequences to the maximum length
                  truncation=True,
                  max_length=2048,
                  return_tensors='pt')

In [ ]:
#DataLoader to iterate over the dataset in batches.
#  Gpu does not permits batch size > 2
tensor_dataset = TensorDataset(pt_batch['input_ids'], pt_batch['attention_mask'])
dataloader = DataLoader(tensor_dataset, batch_size=2, shuffle=True)

In [ ]:
# BitsAndBytes for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,               # loading the model in 4-bit
    bnb_4bit_quant_type="nf4",        # quantization  "nf4" for improved precision
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/610M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,    # task type as causal language modeling
    inference_mode=False,
    r=16,                             # low rank adaptation
    lora_alpha=32,                     # alpha scaling factor
    lora_dropout=0.1,
    bias='none',
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]       # apply lora to query, key, value, output proj
)
model = get_peft_model(base_model, peft_config)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)      # decay learning rate

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): StableLmForCausalLM(
      (model): StableLmModel(
        (embed_tokens): Embedding(50304, 2560)
        (layers): ModuleList(
          (0-31): 32 x StableLmDecoderLayer(
            (self_attn): StableLmSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
             

In [ ]:
# Running single epoch over 10000 rows of data due to memory and time constrains.
num_epochs = 1
for epoch in range(num_epochs):
    total_loss = 0
    for prompt_ids, code_ids in dataloader:
        prompt_ids = prompt_ids.to(device)
        code_ids = code_ids.to(device)

        optimizer.zero_grad()  # reset gradients

        # Forward pass
        output = model(input_ids=prompt_ids, labels=code_ids)
        loss = output.loss
        total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

model.save_pretrained('final_tune_model')

Epoch 1/1, Loss: 18.2879


In [ ]:
model.push_to_hub('final_tune_model')

adapter_model.safetensors:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/keetrap/final_tune_model/commit/b9aa9f3406ab2245ddda946fcf8c00e70d04eb50', commit_message='Upload model', commit_description='', oid='b9aa9f3406ab2245ddda946fcf8c00e70d04eb50', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
peft_model_id = "/kaggle/working/final_tune_model"
config = PeftConfig.from_pretrained(peft_model_id)
# Loading the fine tuned model using the base model and the PEFT config.
model = PeftModel.from_pretrained(base_model, peft_model_id)

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant",
    },
    {
        "role": "user",
        "content": "Generate shopping website for Fashion Brand: A visually stunning layout with a full-width, rotating image carousel showcasing their latest collections, a bold, center-aligned logo, and a bottom navigation menu. The color palette is inspired by the latest fashion trends."
    },
]

prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [ ]:
inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

tokens = model.generate(
    **inputs,
    max_new_tokens=1024,
    temperature=0.5,
    top_p=0.95,
    top_k=100,
    do_sample=True,
    use_cache=True
)

output = tokenizer.batch_decode(tokens[:, inputs.input_ids.shape[-1]:], skip_special_tokens=False)[0]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
print(output)

To generate a shopping website for a fashion brand with the described layout and features, we will use HTML, CSS, and JavaScript. We will also utilize a carousel library to handle the rotating image functionality.

Here's an outline of the steps we'll follow:

1. Create the HTML structure for the website, including the header, main content area, navigation menu, and footer.
2. Style the website using CSS, incorporating a visually stunning layout, a full-width carousel, a bold logo, and a bottom navigation menu.
3. Use JavaScript to handle the rotation functionality of the carousel.

Let's start by creating the HTML structure.
```html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Fashion Brand</title>
  <link rel="stylesheet" href="styles.css">
</head>
<body>
  <header>
    <div class="logo">
      <img src="logo.png" alt="Fashion Brand Logo">
    </div>
  </header>
  <main>
    <div cla